# Inquinamento Airnet

In [1]:
import json
from io import StringIO

import requests
from sseclient import SSEClient
from tqdm import tqdm
import geopandas as gpd

from my_paths import *

## Scraping

Otteniamo gli id delle stazioni di Milano

In [64]:
URL = "https://airnet.waqi.info/airnet/validator/check/112273" # Una stazione a caso di milano\
risposta = requests.get(URL)
risposta_json = risposta.json()
neighbors = risposta_json["data"]["neighbors"]  # Da questa riusciamo a ricavare tutte le stazioni "vicine"

inizializziamo il geojson con id e coordinate delle stazioni

In [65]:
geojson = {
    "type": "FeatureCollection",
    "name": "Inquinamento",
    "crs": {
        "type": "name",
        "properties": {
            "name": "urn:ogc:def:crs:EPSG::6707"
        }
    },
    "features": []
}
for geo, id in zip(neighbors["geos"], neighbors["ids"]):
    if not id == 353767: # Stazione marco emme, Milan, Italy che non contiene dati pm10 e pm25
        geojson["features"].append(
            {
                "type": "Feature",
                "properties": {
                    "id": id,
                    "longitudine": geo[1],
                    "latitudine": geo[0],
                    "name": "",
                    "pm10": [],
                    "pm25": []
                },
                "geometry": {
                    "type": "Point",
                    "coordinates": [geo[1], geo[0]]
                }
            }
        )

Estraiamo i dati di ogni stazione (per i 2 inquinanti) e li inseriamo direttamente tra le "features" >> "properties" del geojson inizializzato

In [66]:
pbar = tqdm(total=len(geojson["features"]), desc="Scraping progress...")

for feature in geojson["features"]:
    for inquinante in ["pm10", "pm25"]:
        url = f"https://airnet.waqi.info/airnet/sse/historic/daily/{feature["properties"]["id"]}?specie={inquinante}"
        response = requests.get(url, params={"specie": inquinante}, headers={"Accept": "text/event-stream"}, stream=True)
        client = SSEClient(response)
        for i, event in enumerate(client.events()):
            if event.event != "error":
                try:
                    data = json.loads(event.data)
                    if i == 0:
                        feature["properties"]["name"] = data["meta"]["name"]
                    elif isinstance(data, dict): #Ogni tanto ci sono righe sporche di stringhe
                        feature["properties"][inquinante].append(data)
                except json.JSONDecodeError:
                    print("Errore JSON:", event.data)
    pbar.update(1)

salvo il file geojson creato in Raw

In [67]:
with open(PATH_INQUINAMENTO_INGESTION_RAW, 'w') as file:
    json.dump(geojson, file, indent=4)

## ETL

A partire dai dati giornalieri dei 2 inquinanti (pm10 e pm25) ricaviamo gli aggregati per anno sottoforma di media delle mediane giornaliere.

In [2]:
def set_mean_pm(row):
    for pm in ["pm10", "pm25"]:
        df = gpd.pd.read_json(StringIO(row[pm]))
        df["day"] = gpd.pd.to_datetime(df["day"])
        df = df[df['day'].dt.year.isin([2025, 2024, 2023, 2022])].reset_index()
        if len(df) > 150:
            row[pm] = df["median"].mean()
        else:
            row[pm] = None
    return row

gdf = gpd.read_file(PATH_INQUINAMENTO_INGESTION_RAW)
gdf = gpd.GeoDataFrame(gdf.apply(set_mean_pm, axis=1), geometry=gdf.geometry.name, crs=CRS_GRAD)

# Eliminiamo la "stazione" che identifica milano in generale, non una vera stazione
gdf = gdf[gdf["id"] != 514987]

Salviamo su Clean

In [ ]:
gdf.to_file(PATH_INQUINAMENTO_INGESTION_CLEAN, driver="GeoJSON")

Aggreghiamo per Municipio, facendo la media delle stazioni all'interno di ogni Municipio (solo pm10)

In [ ]:
gdf = gpd.read_file(PATH_INQUINAMENTO_INGESTION_CLEAN)

In [3]:
colonne_da_tenere = ["pm25", "geometry"]
gdf_inquinamento_municipi = gdf[colonne_da_tenere].copy()
gdf_municipi = gpd.read_file(PATH_MUNICIPI_CLEAN)[["MUNICIPIO", "geometry"]]
gdf_inquinamento_municipi = gpd.sjoin(gdf_inquinamento_municipi, gdf_municipi, "left", predicate="intersects").drop(["index_right"], axis=1)

gdf_inquinamento_municipi.loc[gdf_inquinamento_municipi["MUNICIPIO"].isna(), "MUNICIPIO"] = 6
gdf_inquinamento_municipi["MUNICIPIO"]= gdf_inquinamento_municipi["MUNICIPIO"].astype(int)
gdf_inquinamento_municipi = gdf_inquinamento_municipi.sort_values("MUNICIPIO")

for idx, row in gdf_inquinamento_municipi.iterrows():
    for idx_municipio, row_municipio in gdf_municipi.iterrows():
        if row["MUNICIPIO"] == row_municipio["MUNICIPIO"]:
            geometry = row_municipio["geometry"]
        gdf_inquinamento_municipi.loc[idx, "geometry"] = geometry
gdf_inquinamento_municipi
conteggi = gpd.GeoDataFrame(gdf_inquinamento_municipi.groupby(['MUNICIPIO', 'geometry']).mean().reset_index(), crs=CRS_GRAD)
conteggi.to_file(PATH_INQUINAMENTO_MEDIA_MUNICIPI_CLEAN, driver="GeoJSON")